# EDA

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv("training_data.csv")
df.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
3,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
4,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...


In [3]:
log = {}
log["original length"] = df.shape[0]

In [5]:
print('event types: {}'.format(set(df['event_type'])))
print('product actions: {}'.format(set(df['product_action'])))

event types: {'event_product', 'pageview'}
product actions: {nan, 'add', 'remove', 'detail', 'purchase'}


## Pre-processing

Filtered and modified the data with following conditions:
1. with 'add' in the session
2. cut sessions after the first 'add'
3. take the 'purchase' out of the session
2. keep the session with the length between 5 and 155


### Sessionization

In [6]:
# derive sessions from action by action dataset
# merge all the actions by session_id

df['product_action'] = df['product_action'].fillna('view')
df = df.groupby('session_id_hash')['product_action'].agg(list).reset_index()
df.head()

,session_id_hash,product_action
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, detail, add, view, view, view, view, vi..."
1,000009f36a40de1d557afc083dbb3fc03eef2473337bad...,"[view, view]"
2,00000e812c3076d18245710a31b348d3f23314b7d0dc90...,[view]
3,00001355930ff05e66ab30bccff221c33eba90e1517397...,"[view, detail]"
4,0000162d1dad0beb867c191ab2c8c7c06086cc57d9ebe2...,"[view, view, view, view, detail, view, detail,..."


In [17]:
###################################
### Only keep sessions with 'add'
###################################
# %%timeit
# df.drop(df[df.product_action.map(lambda x: 'add' not in x)].index)

df = df[df.product_action.map(set(['add']).issubset)]

log["contain add"] = df.shape[0]

######################################################################
### add class label to each session (BUY V. NO-BUY)  #################################
######################################################################
df['purchase'] = np.where(df.product_action.map(set(['purchase']).issubset), 1, 0)
df.head()

2.35 s ± 99.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
###################################
### Cut actions after the first add
###################################
df.loc[:,'product_action'] = df['product_action'].map(lambda x: x[x.index('add')+1:])
df.head()                             

#######################################################################
### Cut actions before purchase if there is one
### keep the full sesssion if there is not
######################################################################
df['product_action'] = df['product_action'].map(lambda x: x[0:x.index('purchase')] if 'purchase' in x else x)

######################################################################
### Filtered out outliers which > 155 or < 5.    #####################
######################################################################
df["len"] = df["product_action"].map(len)

# removing very short and very long sessions
df.drop(df[df.len < 5].index, inplace=True)
df.drop(df[df.len > 155].index, inplace=True)

log["remove outliers"]=df.shape[0]


,session_id_hash,product_action,purchase
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, view, view, view, view, view, view, det...",0
37,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"[view, detail, view, view, view, detail, view,...",0
64,00010d84aca1294479304044207fd268f63228844779c6...,"[view, view, view, view, view, view, view, vie...",0
84,0001368d732951035a7ef7ef42b345a5c50b7d66966749...,"[view, view, detail, add, view, view, view, vi...",0
119,0001c180fb742f96ff388ba8f67a568e6fa66aed30d0d2...,"[remove, view, detail, view, view, view, remov...",1


### Symbolization

In [26]:
from collections import Counter

sessions = df['product_action'].to_list()
labels = df['purchase'].to_list()

counts = Counter([item for session in sessions for item in session])
# I'm not going to use it, but I'm saving the 0 for padding sessions when symbolising
symbol2idx = {symbol: idx for idx, symbol in enumerate(sorted(counts, key=counts.get, reverse=True), 1)}


df["session"]=df["product_action"].map(lambda session: [symbol2idx[s] for s in session])

{'view': 1, 'detail': 2, 'remove': 3, 'add': 4}


In [31]:
df.head()

,session_id_hash,product_action,purchase,len,session
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, view, view, view, view, view, view, det...",0,15,"[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1]"
37,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"[view, detail, view, view, view, detail, view,...",0,120,"[1, 2, 1, 1, 1, 2, 1, 2, 1, 4, 1, 1, 1, 1, 2, ..."
64,00010d84aca1294479304044207fd268f63228844779c6...,"[view, view, view, view, view, view, view, vie...",0,24,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 2, 1, ..."
84,0001368d732951035a7ef7ef42b345a5c50b7d66966749...,"[view, view, detail, add, view, view, view, vi...",0,13,"[1, 1, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
119,0001c180fb742f96ff388ba8f67a568e6fa66aed30d0d2...,"[remove, view, detail, view, view, view, remov...",1,23,"[3, 1, 2, 1, 1, 1, 3, 3, 3, 1, 1, 3, 1, 3, 1, ..."


### Subset data （5，10，15）

In [68]:
## Subset with sessionn more than 15 post first add to cart
df_15 = df[df.len >= 15]
df_15["session"] = df_15.session.map(lambda x: tuple(x[0:15]))
df_15 = df_15[["session", "purchase"]]

log["more than 15"] = df_15.shape[0]
log

/var/folders/dt/552dh9wd7yx6f5wxdyrg4ghw0000gn/T/ipykernel_68972/3985804867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_15["session"] = df_15.session.map(lambda x: tuple(x[0:15]))


In [71]:
## Subset with sessionn more than 10 post first add to cart
df_10 = df[df.len >= 10]
df_10["session"] = df_10.session.map(lambda x: tuple(x[0:10]))
df_10 = df_10[["session", "purchase"]]

log["more than 10"] = df_10.shape[0]
log

/var/folders/dt/552dh9wd7yx6f5wxdyrg4ghw0000gn/T/ipykernel_68972/3190199024.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10["session"] = df_10.session.map(lambda x: tuple(x[0:10]))


{'original length': 30456445,
 'contain add': 174054,
 'remove outliers': 108626,
 'more than 15': 56071,
 'more than 10': 76710}

In [74]:
## Subset with sessionn more than 5 post first add to cart
df_5 = df[df.len >= 5]
df_5["session"] = df_5.session.map(lambda x: tuple(x[0:5]))
df_5 = df_5[["session", "purchase"]]

log["more than 5"] = df_5.shape[0]
log

{'original length': 30456445,
 'contain add': 174054,
 'remove outliers': 108626,
 'more than 15': 56071,
 'more than 10': 76710,
 'more than 5': 108626}

## Oracle model

In [1]:
import pandas as pd
df = pd.read_csv("data/filtered_data.csv")

In [22]:
def convert_session(x):
    return [int(i) for i in x.strip("[]").split(",")]

df.session = df.session.map(convert_session)

In [23]:
df

,session_id_hash,product_action,purchase,len,session
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"['view', 'view', 'view', 'view', 'view', 'view...",0,15,"[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1]"
1,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"['view', 'detail', 'view', 'view', 'view', 'de...",0,120,"[1, 2, 1, 1, 1, 2, 1, 2, 1, 4, 1, 1, 1, 1, 2, ..."
2,00010d84aca1294479304044207fd268f63228844779c6...,"['view', 'view', 'view', 'view', 'view', 'view...",0,24,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 2, 1, ..."
3,0001368d732951035a7ef7ef42b345a5c50b7d66966749...,"['view', 'view', 'detail', 'add', 'view', 'vie...",0,13,"[1, 1, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,0001c180fb742f96ff388ba8f67a568e6fa66aed30d0d2...,"['remove', 'view', 'detail', 'view', 'view', '...",1,23,"[3, 1, 2, 1, 1, 1, 3, 3, 3, 1, 1, 3, 1, 3, 1, ..."
...,...,...,...,...,...
108621,fffccaf02659cd209eb5504ba532bbdabb712f830f3f51...,"['detail', 'view', 'view', 'remove', 'add', 'r...",0,14,"[2, 1, 1, 3, 4, 3, 1, 1, 1, 1, 1, 1, 2, 1]"
108622,fffdbbaa799c83d83ac6c2666babe7eb2a049320a929e0...,"['view', 'view', 'view', 'view', 'view', 'view...",0,11,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
108623,fffe474ed9f207fc66891586fb001542420e985510e447...,"['view', 'detail', 'view', 'view', 'view', 'vi...",1,30,"[1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 4, ..."
108624,ffff2f2d1dbd19956c340e4e7df3ea2481695387755f4c...,"['view', 'view', 'view', 'view', 'view', 'view...",0,12,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [24]:
def oracle(X_train, y_train):
    '''
    This function return a oracle model based on the session and label.
    '''
    from collections import defaultdict

    d = defaultdict(lambda: defaultdict(int))
    for s, l in zip(X_train, y_train):
        d[str(tuple(s))][l] += 1
        
    model = {}
    for session in d.keys():
        n_buy, n_ws = 0, 0
        for label in d[session].keys():
            if label:
                n_buy += d[session][label]
            else:
                n_ws += d[session][label]
        model[session] = n_buy / (n_buy + n_ws)
        
    return model

def oracle_predict(X_train, X_test, y_train, y_test=None):
    '''
    1. First, get the oracle model from X_train and y_train.
    2. Make prediction on X_test.
    3. For possibility >0.5 predict 1, else 0.
    4. reutrn prediction and F1 score
    '''
    from sklearn import metrics
    model = oracle(X_train, y_train)

    y_pred = []
    for x in X_test:
        pos = model.get(str(x), "#") ## For sessions that not in the model, we assign it to #, and then predict 0.
        y_pred.append(pos)
        
    ## count "#" first and substitute with 0
    missing_rate = y_pred.count("#")/len(y_pred)

    y_pred = [1 if y != "#" and y >= 0.5 else 0 for y in y_pred]
        
    if y_test is not None:
#         y_test = y_test.to_list()
        f1 = metrics.f1_score(y_test, y_pred)
        return(f1, missing_rate,y_pred)
    else:
        return y_pred

In [25]:
log = {}

In [26]:
## make the prediction for 5, 10 and 15, and store the f1 it in a dict
## First split the train/test data
from sklearn.model_selection import train_test_split

for i in [5, 10, 15]:
    
    ## Subset from the original df
    data = df[df.len >= i]
    data["session"] = data.session.map(lambda x: tuple(x[0:i]))
    data = data[["session", "purchase"]]
    
    ## Split train and test
    X_train, X_test, y_train, y_test = train_test_split(data["session"], data["purchase"], test_size=0.2, random_state=123)
    f1, missing_rate, prediction = oracle_predict(X_train, X_test, y_train, y_test)
    log[f"oracle_{i}_f1"] = f1
    log[f"oracle_{i}_missing"] = missing_rate
    
log

/var/folders/dt/552dh9wd7yx6f5wxdyrg4ghw0000gn/T/ipykernel_95125/3251124556.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["session"] = data.session.map(lambda x: tuple(x[0:i]))
/var/folders/dt/552dh9wd7yx6f5wxdyrg4ghw0000gn/T/ipykernel_95125/3251124556.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["session"] = data.session.map(lambda x: tuple(x[0:i]))


{'oracle_5_f1': 0.13331155040026138,
 'oracle_5_missing': 0.0005523336095001381,
 'oracle_10_f1': 0.1893939393939394,
 'oracle_10_missing': 0.05644635640724808,
 'oracle_15_f1': 0.18643216080402011,
 'oracle_15_missing': 0.38831921533660274}

### Error Analysis on Oracle Model

By comparing the F1 of the oracle model applied to 5, 10 and 15-click-session. We can find that the f1 score decreases as the session gets longer. And one explanation can be that as the session length grows, there will be more sessions in test data that cannot be found in the oracle model.

## Save current result

In [27]:
## save log to a json file
import json

json.dump(log, open("data/oracle.json", "w"))
# df.to_csv("data/filtered_data.csv", index=False)

In [225]:
!tree

.
├── EDA.ipynb
├── README.md
├── SharedTask.ipynb
├── data
│   ├── filtered_data.csv
│   └── log.json
├── early_prediction.ipynb
├── filtered_data.csv
└── training_data.csv

1 directory, 8 files
